In [1]:
%matplotlib inline
#%pylab
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mpc
import matplotlib.dates as dts
import numpy as np
import math
#import pandas as pd
import itertools
import os
#import ROOT
import datetime
#from root_numpy import root2array, root2rec, tree2rec, array2root
from scipy.optimize import curve_fit
from scipy.misc import factorial
plt.rcParams.update({'font.size': 16})

In [2]:
from decode import GetWaveforms
from decode import GetWaveformsNoiseRemoval

In [3]:
from datetime import datetime

In [4]:
def gettime(name):
    date = name.split('PrM1-')[-1].split('.dat')[0]
    time = datetime.strptime(date,'%Y_%m_%d-%H_%M_%S')
    return time

In [5]:
def QaQc(wf_v):
    anode_v = wf_v[0]#[700:]
    cathode_v = wf_v[1]#[500:]
    Qa = np.amax(anode_v)
    Qc = np.amin(cathode_v)
    return Qa,Qc

def QaQcSTATS(ev_wf_v):
    
    Qa_v = []
    Qc_v = []
    for wf_v in ev_wf_v:
        Qa,Qc = QaQc(wf_v)
        Qa_v.append(Qa*1e3)
        Qc_v.append(Qc*1e3)
    Qa_v = np.array(Qa_v)
    Qc_v = np.array(Qc_v)

    Qa_avg = np.mean(Qa_v)
    Qa_rms = np.std(Qa_v)
    Qc_avg = np.mean(Qc_v)
    Qc_rms = np.std(Qc_v)
    
    return [[Qa_avg,Qa_rms],[Qc_avg,Qc_rms]]




def mean_plot(measures,Nsample):
    Nmeasures = 0
    N = len(measures)
    
    #mean_measures_anode=np.zeros(len(measures[0][0]))
    #mean_measures_cathode=np.zeros(len(measures[0][0]))
    mean_measures_anode=np.zeros(Nsample)
    mean_measures_cathode=np.zeros(Nsample)
    for index in range(N):
        # Find "noise peak"
        NoiseTick = 0
        CurrentTick = 0
        while ((NoiseTick == 0) and (CurrentTick < len(measures[index][0])) ):
            #print CurrentTick
            if measures[index][0][CurrentTick] > 0.2: # 0.2 is amplitude of "noise tick"
                NoiseTick = CurrentTick
            CurrentTick += 1
        # if we don't find the noise tick -> quit
        if (NoiseTick * 0.0005 > 0.2):
            continue
        #print(index)
        mean_measures_anode=mean_measures_anode+measures[index][0][NoiseTick:NoiseTick+Nsample]
        mean_measures_cathode=mean_measures_cathode+measures[index][1][NoiseTick:NoiseTick+Nsample]
        Nmeasures += 1
        
    mean_measures_anode=mean_measures_anode/Nmeasures
    mean_measures_cathode=mean_measures_cathode/Nmeasures
    
    return [mean_measures_anode,mean_measures_cathode]

def det_baseline(values_vector,time_vector):
    baseline_anode=0.
    baseline_cathode=0.
    for i in range(len(time_vector)):
        if time_vector[i]<0.15:
            baseline_anode=baseline_anode+values_vector[0][i]
            baseline_cathode=baseline_cathode+values_vector[1][i]
            counts=i
    baseline_anode=baseline_anode/counts
    baseline_cathode=baseline_cathode/counts
    return baseline_anode,baseline_cathode

def time_drift1(values_vector, time_vector):
        tol=1
        time_anode=[]
        time_cathode=[]
        halfheight_values_anode=[]
        halfheight_values_cathode=[]
        for n in range(len(values_vector)):
            control_anode=0
            control_cathode=0
            baseline_anode,baseline_cathode=det_baseline(values_vector[n],time_vector)
            
            #print(baseline_anode,baseline_cathode)
            max_anode=np.max(values_vector[n][0])
            min_cathode=np.min(values_vector[n][1])
            #print (min_cathode, baseline_cathode)
            
            for m in range(len(values_vector[n][0])):
                
                if control_anode==0:
                    if values_vector[n][0][m]>((max_anode-baseline_anode)/2.0-tol) and values_vector[n][0][m]<((max_anode-baseline_anode)/2+tol):
                        time_anode.append(time_vector[m])
                        halfheight_values_anode.append(values_vector[n][0][m])
                        control_anode=1
                if control_cathode==0:
                    #print(values_vector[n][1][m],(min_cathode-baseline_cathode)/2 )
                    
                    if values_vector[n][1][m]>((min_cathode-baseline_cathode)/2-tol) and values_vector[n][1][m]<((min_cathode-baseline_cathode)/2+tol):
                        time_cathode.append(time_vector[m])
                        halfheight_values_cathode.append(values_vector[n][1][m])
                        control_cathode=1   
                        #print(values_vector[n][1][m]) 
            if(control_anode==0 or control_cathode==0):
                print('errore')
            
        time_dr=np.array(time_anode)-np.array(time_cathode)
        return(time_dr,time_anode,time_cathode,halfheight_values_anode,halfheight_values_cathode)
    
    

In [6]:
def fitlin(values_vector,time_vector):
    tol=0.005
    control_anode_min=0
    control_anode_max=0
    control_cathode_min=0
    control_cathode_max=0
    baseline=det_baseline(values_vector,time_vector)
    baseline_anode=baseline[0]
    baseline_cathode=baseline[1]
    
    max_anode=np.max(values_vector[0])
    min_cathode=np.min(values_vector[1])
    
    for m in range(len(values_vector[0])):
        if control_anode_min==0 and values_vector[0][m]>=baseline_anode+tol:
            control_anode_min=1
            index_anode_min=m
        if control_anode_max==0 and values_vector[0][m]+tol>=max_anode:
            control_anode_max=1
            index_anode_max=m
        if control_cathode_min==0 and values_vector[1][m]<=baseline_cathode-tol:
            control_cathode_min=1
            index_cathode_min=m
        if control_cathode_max==0 and values_vector[1][m]-tol<=min_cathode:
            control_cathode_max=1
            index_cathode_max=m       
    an=np.polyfit(time_vector[index_anode_min:index_anode_max], values_vector[0][index_anode_min:index_anode_max],1)
    cat=np.polyfit(time_vector[index_cathode_min:index_cathode_max], values_vector[1][index_cathode_min:index_cathode_max],1)
    an = np.poly1d(an)   
    cat=np.poly1d(cat)
    residuals_anode=[np.array(an(time_vector[index_anode_min:index_anode_max]))-np.array(values_vector[0][index_anode_min:index_anode_max]),time_vector[index_anode_min:index_anode_max]]
    residuals_cathode=[np.array(cat(time_vector[index_cathode_min:index_cathode_max]))-np.array(values_vector[1][index_cathode_min:index_cathode_max]),time_vector[index_cathode_min:index_cathode_max]]
    return(an,cat,residuals_anode,residuals_cathode)

def time_drift(values_vector, time_vector):
        tol=0.0005
        time_anode=[]
        time_cathode=[]
        halfheight_values_anode=[]
        halfheight_values_cathode=[]
        
        control_anode=0
        control_cathode=0
        baseline_anode,baseline_cathode=det_baseline(values_vector,time_vector)
            
            #print(baseline_anode,baseline_cathode)
        max_anode=np.max(values_vector[0])
        min_cathode=np.min(values_vector[1])
            #print (min_cathode, baseline_cathode)
        #print((max_anode-baseline_anode)/2.0,(min_cathode-baseline_cathode)/2.0, baseline_anode, baseline_cathode, max_anode, min_cathode)    
        for m in range(len(values_vector[0])):
            if control_anode==0:
                if (values_vector[0][m]>(((max_anode-baseline_anode)/2.0)-tol) and values_vector[0][m]<(((max_anode-baseline_anode)/2.0)+tol)):
                    time_anode.append(time_vector[m])
                    halfheight_values_anode.append(values_vector[0][m])
                    control_anode=1
            if control_cathode==0:
                    #print(values_vector[n][1][m],(min_cathode-baseline_cathode)/2 )
                    
                if (values_vector[1][m]>(((min_cathode-baseline_cathode)/2.0)-tol) and values_vector[1][m]<(((min_cathode-baseline_cathode)/2.0)+tol)):
                    time_cathode.append(time_vector[m])
                    halfheight_values_cathode.append(values_vector[1][m])
                    control_cathode=1   
                        #print(values_vector[n][1][m]) 
        print((max_anode-baseline_anode)/2.0-halfheight_values_anode[0])
        time_dr=np.array(time_anode)-np.array(time_cathode)
        return(time_dr,time_anode,time_cathode,halfheight_values_anode,halfheight_values_cathode)
    
def time_drift_line(values_vector, time_vector, an, cat):
        tol=0.0005
        time_anode=[]
        time_cathode=[]
        control_anode=0
        control_cathode=0
        baseline_anode,baseline_cathode=det_baseline(values_vector,time_vector)
          
        for m in range(len(values_vector[0])):
            if control_anode==0:
                if (an(time_vector[m])>(baseline_anode-tol) and an(time_vector[m])<(baseline_anode+tol)):
                    time_anode.append(time_vector[m])
                    control_anode=1
            if control_cathode==0:
                if (cat(time_vector[m])>(baseline_cathode-tol) and cat(time_vector[m])<(baseline_cathode+tol)):
                    time_cathode.append(time_vector[m])
                    control_cathode=1   
                        
        
        time_dr=np.array(time_anode)-np.array(time_cathode)
        return(time_dr,time_anode,time_cathode)
    

    
def time_drift_anode(values_vector, time_vector):
        tol=0.005#0.0003 #per run 1 0.00034
        #per gas 0.005
        
        control_anode=0
        control_cathode=0
        baseline_anode,baseline_cathode=det_baseline(values_vector,time_vector)
            
            #print(baseline_anode,baseline_cathode)
        max_anode=np.max(values_vector[0])
        ind_max=values_vector[0].tolist().index(np.max(values_vector[0]))
        time_max_anode=time_vector[ind_max]#[values_vector[0].index(np.max(values_vector[0]))]
        ind_min=values_vector[1].tolist().index(np.min(values_vector[1]))
        
        #print(ind_max,ind_min)
        min_cathode=np.min(values_vector[1])
        time_min_cathode=time_vector[ind_min]#[values_vector[1].index(np.min(values_vector[1]))]
            #print (min_cathode, baseline_cathode)
        #print((max_anode-baseline_anode)/2.0,(min_cathode-baseline_cathode)/2.0, baseline_anode, baseline_cathode, max_anode, min_cathode)    
        for m in range(len(values_vector[0])):
            if control_anode==0:
                if (values_vector[0][m]>(((max_anode-baseline_anode)/2.0)-tol) and values_vector[0][m]<(((max_anode-baseline_anode)/2.0)+tol)):
                    time_anode=time_vector[m]
                    halfheight_values_anode=values_vector[0][m]
                    control_anode=1
                    print("anodo")
                    print(halfheight_values_anode, (max_anode-baseline_anode)/2.0)
            if control_cathode==0:
                    #print(values_vector[n][1][m],(min_cathode-baseline_cathode)/2 )
                    
                if (values_vector[1][m]>(((min_cathode-baseline_cathode)/2.0)-tol) and values_vector[1][m]<(((min_cathode-baseline_cathode)/2.0)+tol)):
                    time_cathode=time_vector[m]
                    halfheight_values_cathode=values_vector[1][m]
                    control_cathode=1   
                    print("catodo")
                    print(halfheight_values_cathode,(min_cathode-baseline_cathode)/2.0) 
        
        time_dr_anode=-(time_anode-time_max_anode)
        time_dr_cathode=-(time_cathode-time_min_cathode)
        return(time_dr_anode,time_dr_cathode,time_anode,time_cathode,halfheight_values_anode,halfheight_values_cathode)

In [ ]:
PATH = '/Users/davidc/Downloads/RUN3/7psiC51VCG61V/data/'
datafile_v = os.listdir(PATH)
print(datafile_v)
dt = 0.0005
datafile_v.sort()
print(datafile_v)
RC=0.240

In [ ]:
datafile = PATH + datafile_v[-1]
ev_wf_v = GetWaveformsNoiseRemoval(datafile)

In [ ]:
time = gettime(datafile_v[-1])
dt = 0.0005

time_v = np.linspace(0,dt*len(ev_wf_v[0][0]),len(ev_wf_v[0][0]))


time_dri,time_anode,time_cathode,FWHM_values_anode,FWHM_values_cathode=time_drift1(ev_wf_v,time_v)
#print(len(time_dr),len(time_anode),len(time_cathode),len(FWHM_values_anode),len(FWHM_values_cathode))
print(len(ev_wf_v))
for n in xrange(len(ev_wf_v)):
    
    an,cat,res_an,res_cat=fitlin(ev_wf_v[n],time_v)
    c,d,e=time_drift_line(ev_wf_v[n], time_v, an, cat)
    max_anode=np.max(ev_wf_v[n][0])
    ind_max=ev_wf_v[n][0].tolist().index(np.max(ev_wf_v[n][0]))
    time_max_anode=time_v[ind_max]#[values_vector[0].index(np.max(values_vector[0]))]
    ind_min=ev_wf_v[n][1].tolist().index(np.min(ev_wf_v[n][1]))
        
        #print(ind_max,ind_min)        
    min_cathode=np.min(ev_wf_v[n][1])
    time_min_cathode=time_v[ind_min]
    fig = plt.figure(1)
    frame1=fig.add_axes((.1,.3,.8,.6))
    plt.plot(time_v,ev_wf_v[n][0],color='r',label='Anode')
    plt.plot(time_v,np.abs(ev_wf_v[n][1]),color='b',label='Cathode')
    plt.plot(time_cathode[n],FWHM_values_cathode[n],color='g',marker='o')
    plt.plot(time_v,an(time_v),color='c')
    plt.plot(time_v,cat(time_v),color='m')
    plt.plot(time_anode[n],FWHM_values_anode[n],color='g',marker='o')
    plt.plot(time_max_anode,max_anode,color='g',marker='o')
    plt.plot(time_min_cathode,np.abs(min_cathode),color='g',marker='o')
    #frame1.set_xticklabels([])
    plt.grid()
    plt.legend(loc=1)
    
    plt.xlabel('Time [ms]',fontsize=20)
    plt.ylabel('Amplitude [mV]',fontsize=20)
   # plt.title('Time : %s'%time)
    plt.ylim([0.0,0.5])
    #plt.xlim([0.2,0.26])
    plt.ylim([0.0,0.12])
    #frame2=fig.add_axes((.1,.1,.8,.2))        
    #plt.plot(res_cat[1],res_cat[0],'or')
    #plt.xlim([0.2,0.3])
    
    #plt.grid()
    plt.show()
    

In [ ]:
date_v = []
Qa_avg_v_run2 = []
Qa_rms_v_run2 = []
Qc_avg_v_run2 = []
Qc_rms_v_run2 = []
Qa_baseline=[]
Qc_baseline=[]
time_drift_mean=[]
time_drift_mean_line=[]
time_drift_rms=[]
time_drift_rms_line=[]
time_drift_mean_anodeR2=[]
time_drift_rms_anode=[]
time_drift_mean_cathodeR2=[]
time_drift_rms_cathode=[]
e_charge_anode_v_run2=[]
e_charge_cathode_v_run2=[]
err_charge_anode_v_run2=[]
err_charge_cathode_v_run2=[]
measure=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
num=0
for datafile in datafile_v:
    if datafile!='.DS_Store':
        num=num+1
        print(datafile)
        if (datafile.find('.dat') < 0): continue
        time = gettime(datafile)
        fin = PATH+datafile
        try:
            ev_wf_v = GetWaveformsNoiseRemoval(fin)
            
            a=mean_plot(ev_wf_v,len(ev_wf_v[0][0]))
            
            time_v = np.linspace(0,dt*len(ev_wf_v[0][0]),len(ev_wf_v[0][0]))
            
            #time_dr,time_anode,time_cathode,FWHM_values_anode,FWHM_values_cathode=time_drift(a,time_v)
            time_dr_anode,time_dr_cathode,time_anode,time_cathode,half_anode,half_cathode=time_drift_anode(a, time_v)
            print('times=%s'%[time_dr_anode,time_dr_cathode])
            #print(time_dr,time_anode,time_cathode,FWHM_values_anode,FWHM_values_cathode)
            #an,cat,res_an,res_cat=fitlin(a,time_v)
            #c,d,e=time_drift_line(a, time_v, an, cat)
            
            
            
            
            
            corr_anode=((1.-math.exp(-2*time_dr_anode/RC)))*RC/(2*time_dr_anode)
            
            print('correzione anodo=%s'%[2*time_dr_anode,corr_anode])#time_dr_anode e' la meta' del time drift che ci interessa
            corr_cathode=((1.-math.exp(-2*time_dr_cathode/RC)))*RC/(2*time_dr_cathode)
            
            print('correzione catodo=%s'%[2*time_dr_cathode,corr_cathode])
            
            
            
            
            
            
            fig = plt.figure(1)
            frame1=fig.add_axes((.1,.3,.8,.6))
            plt.grid()
            plt.plot(time_v,a[0],color='r',label='Anode')
            plt.plot(time_v,np.abs(a[1]),color='b',label='Cathode')
            
            
            #plt.plot(time_cathode[0],FWHM_values_cathode[0],color='g',marker='o')
            #plt.plot(e[0],cat(e[0]),color='y',marker='o')
            #plt.plot(d[0],an(d[0]),color='y',marker='o')
            #plt.plot(time_v,an(time_v),color='c')
            #plt.plot(time_v,cat(time_v),color='m')
            #plt.plot(time_anode[0],FWHM_values_anode[0],color='g',marker='o')
            plt.plot(time_cathode,half_cathode,color='g',marker='o')
            plt.plot(time_anode,half_anode,color='g',marker='o')
            #frame1.set_xticklabels([])
            
            plt.legend(loc=1)
    
            plt.xlabel('Time [ms]',fontsize=20)
            plt.ylabel('Amplitude [mV]',fontsize=20)
   
            #plt.ylim([-0.04,0.04])
            #plt.xlim([0.,0.6])
        
            
            #frame2=fig.add_axes((.1,.1,.8,.2))        
            #plt.plot(res_cat[1],res_cat[0],'ob')
            #plt.plot(res_an[1],res_an[0],'or')
            plt.xlim([0.2,0.3])
            #plt.ylim([-0.01,0.01])
            #plt.grid()
            fig.savefig('1method_plot%s.png'%num)
            plt.show()
            anode,cathode=det_baseline(a,time_v)
            anode=anode*1000
            cathode=cathode*1000
            Qa_baseline.append(anode)
            Qc_baseline.append(cathode)
            res = QaQcSTATS(ev_wf_v)
            Qa_avg = res[0][0]
            Qa_rms = res[0][1]
            Qc_avg = res[1][0]
            Qc_rms = res[1][1]
            
            date_v.append(time)
            volt_anode=(np.max(a[0])*1000-anode)/corr_anode
            volt_cathode=(np.min(a[1])*1000-cathode)/corr_cathode
            Qa_avg_v_run2.append(volt_anode)
            Qc_avg_v_run2.append(volt_cathode)
            print '%.02f +/- %.02f mV Qc : %.02f +/- %.02f mV'%(Qa_avg_v[-1],Qa_rms,Qc_avg_v[-1],Qc_rms)
            print(time_v[a[0].tolist().index(np.max(a[0]))])
            Qa_rms_v_run2.append(0)#(Qa_rms/np.sqrt(len(ev_wf_v)))
            Qc_rms_v_run2.append(0)#(Qc_rms)
            time_drift_mean_anodeR2.append(time_dr_anode)
            #time_drift_rms_anode.append(0)
            time_drift_mean_cathodeR2.append(time_dr_cathode)
            #time_drift_rms_cathode.append(0)
            #time_drift_mean_line.append(c[0])
            #time_drift_rms_line.append(np.std(c))
            e_charge_anode=volt_anode*(10**-14)/(1.602*(10**-19)) #10^-14 viene da 10 picofahrrad*10^-12 per trasformarli in faahrrad, per 10^-3 per i millivolt
            e_charge_cathode=volt_cathode*(10**-14)/(1.602*(10**-19))
            e_charge_anode_v_run2.append(e_charge_anode)
            e_charge_cathode_v_run2.append(e_charge_cathode)
            err_charge_anode_v_run2.append(np.sqrt(e_charge_anode))
            err_charge_cathode_v_run2.append(np.sqrt(np.abs(e_charge_cathode)))
            
        except:
            print 'failure @ file %s'%datafile
        
#date_v = matplotlib.dates.date2num(date_v)
Qa_mean = np.mean(Qa_avg_v)
Qa_error = np.std(Qa_avg_v)
Qc_mean = np.mean(Qc_avg_v)
Qc_error = np.std(Qc_avg_v)
print(Qa_mean, Qa_error, Qc_mean, Qc_error, Qa_baseline, Qc_baseline)

In [ ]:
(7.6937309659381325, 1.1256126041983856, -7.7590585340434739, 0.54232149664105656, [2.7978104658523533, 2.6245828035439529, 2.5039480075125962, 2.6903032606873829, 2.5922686603131178, 2.5436450936437911, 2.5182126237409634, 2.3909177020846943, 2.4994919952026535, 2.5316899089608986], [3.698202930977637, 3.5271891226048013, 3.4465753528800951, 3.6311729825964254, 3.5315187760215294, 3.4232119356027284, 3.3587175501138904, 3.2874217209546348, 3.5148576278107377, 3.3513330475285681])

fin = PATH+'PrM1-2018_08_07-17_19_48.dat'
time = gettime('PrM1-2018_08_07-17_19_48.dat')
ev_wf_v = GetWaveformsNoiseRemoval(fin)
Nsamples = 2000


a=mean_plot_noise_removal(ev_wf_v) 
dt = 0.0005
time_v = np.linspace(0,dt*len(ev_wf_v[0][0]),len(ev_wf_v[0][0]))
fig = plt.figure(figsize=(8,5))
plt.plot(time_v,a[0],color='r',label='Anode')
plt.plot(time_v,a[1],color='b',label='Cathode')
plt.grid()
plt.legend(loc=1)
plt.xlabel('Time [ms]',fontsize=20)
plt.ylabel('Amplitude',fontsize=20)
   # plt.title('Time : %s'%time)
plt.ylim([-0.004,0.01])
    #plt.xlim([0,0.05])
plt.show()

b=mean_plot(ev_wf_v,Nsamples) 
dt = 0.0005
time_v = np.linspace(0,dt*len(ev_wf_v[0][0]),len(ev_wf_v[0][0]))
fig = plt.figure(figsize=(8,5))
plt.plot(time_v[:Nsamples],b[0],color='r',label='Anode')
plt.plot(time_v[:Nsamples],b[1],color='b',label='Cathode')
plt.grid()
plt.legend(loc=1)
plt.xlabel('Time [ms]',fontsize=20)
plt.ylabel('Amplitude',fontsize=20)
   # plt.title('Time : %s'%time)
plt.ylim([-0.004,0.01])
    #plt.xlim([0,0.05])
plt.show()
anode,cathode=det_baseline(a,time_v)
print(anode,cathode)



In [ ]:
fig = plt.figure(figsize=(16,6))
frame1=fig.add_axes((.1,.3,.8,.6))
measure1=[1]
#anode_voltages=[30,60,100,150,200,250,300,350,400,450]
anode_voltages=[46,75,100,150,200,250,300,350,400,450,500,600,700,800,900,1000]#RUN2
#anode_voltages=[46,75,150,250,350,450,550,650,750,850,950]
cathode_voltages=[62.5,83,103,123,143,200,250,300,350,400,450,500,600,700]
anode_voltages_run1=[13,18,23,34,44,54,103,150,200,300,400,491,600,700,800,900,1000]#RUN1
#plt.plot_date(date_v,np.abs(Qa_avg_v))
#plt.plot_date(date_v,np.abs(Qc_avg_v))
time_drift_rms_anodeR1=np.zeros(len(anode_voltages_run1))
Qa_avg_v=np.array(Qa_avg_v)
Qc_avg_v=np.array(Qc_avg_v)
Qa_rms_v=np.array(Qa_rms_v)
Qc_rms_v=np.array(Qc_rms_v)
ratio=(Qa_avg_v)/(Qc_avg_v)
print(ratio)
err_ratio=np.sqrt((Qa_rms_v/Qc_avg_v)**2+(Qc_rms_v*Qa_avg_v/(Qc_avg_v)**2)**2)
print(err_ratio)

#plt.errorbar(anode_voltages_run1,np.abs(e_charge_anode_v_run1),yerr=err_charge_anode_v_run1,fmt='.',color='r',label='Anode run 1')
#plt.errorbar(anode_voltages_run1,np.abs(e_charge_cathode_v_run1),yerr=err_charge_cathode_v_run1,fmt='.',color='b',label='Cathode run 1')
#plt.errorbar(anode_voltages,np.abs(e_charge_anode_v_run2),yerr=err_charge_anode_v_run2,fmt='.',color='m',label='Anode run 2')
#plt.errorbar(anode_voltages,np.abs(e_charge_cathode_v_run2),yerr=err_charge_cathode_v_run2,fmt='.',color='c',label='Cathode run 2')
plt.plot(anode_voltages_run1,np.abs(Qa_avg_v_run1),'o',color='r',label='Anode run 1')#yerr=Qa_rms_v,
plt.plot(anode_voltages_run1,np.abs(Qc_avg_v_run1),'o',color='b',label='Cathode run 1')#yerr=Qc_rms_v,
plt.plot(anode_voltages,np.abs(Qa_avg_v_run2),'o',color='m',label='Anode run 2')#yerr=Qa_rms_v,
plt.plot(anode_voltages,np.abs(Qc_avg_v_run2),'o',color='c',label='Cathode run 2')#yerr=Qc
#plt.errorbar(anode_voltages,np.abs(ratio),yerr=err_ratio,fmt='o',color='g')
#plt.errorbar(anode_voltages_run1,2*np.array(time_drift_mean_anode),yerr=time_drift_rms_anodeR1,fmt='o',color='m',label='Anode run 1')
#plt.errorbar(anode_voltages,2*np.array(time_drift_mean_anodeR2),yerr=time_drift_rms_anode,fmt='o',color='y',label='Anode run 2')
#plt.errorbar(anode_voltages_run1,2*np.array(time_drift_mean_cathode),yerr=time_drift_rms_anodeR1,fmt='o',color='b',label='Cathode run 1')

#plt.errorbar(anode_voltages,2*np.array(time_drift_mean_cathodeR2),yerr=time_drift_rms_cathode,fmt='o',color='k',label='Cathode run 2')


#plt.ylim([700000,2000000])

plt.xlim([0,1100])
plt.grid()
plt.legend(numpoints=1,fontsize=18, loc='lower right')
plt.xlabel('Anode voltage [V]',fontsize=20)
plt.ylabel('Amplitude [mV] [50-run avg.]',fontsize=20)
#plt.ylabel('Charge [number of e-]',fontsize=20)
#plt.ylabel('Amplitude anode / amplitude cathode',fontsize=15)
#plt.ylabel('Drift time',fontsize=20)
#plt.title('Drift time in LAr with Vc=-100 V Vcg=-65 V in Run 2',fontsize=30)
plt.title('Measures in LAr with Vc=-200 V Vcg=-130 V in Run 1 and Run 2',fontsize=25)
#frame2=fig.add_axes((.1,.1,.8,.2))   
#diff=np.array(time_drift_mean)-np.array(time_drift_mean_line)
#plt.plot(anode_voltages,diff,'oy')
#plt.xlim([0,1100])        
#plt.grid()
#fig.savefig('LAr_corrected_run1.png')
#fig.savefig('LAr_charge_run1.png')
fig.savefig('LAr_signals_compared.png')
plt.show()

In [ ]:
print(Qc_avg_v, Qc_rms_v, Qa_avg_v, Qa_rms_v)

In [ ]:
Qc_max=[] 
Qc_err=[] 
Qa_max=[]
Qa_err=[]
Qa_media=[]
Qa_stadev=[]
Qc_media=[]
Qc_stadev=[]
Fiber_n=[1,2,3,4,5,6,7,8,9]
print(len(Qc_max), len(Qc_err), len(Qa_max), len(Qa_err), len(Fiber_n))

In [ ]:
Qc_max.append(min(Qc_avg_v))
Qc_err.append(Qc_rms_v[Qc_avg_v.index(min(Qc_avg_v))])
Qa_max.append(max(Qa_avg_v))
Qa_err.append(Qa_rms_v[Qa_avg_v.index(max(Qa_avg_v))])
Qa_media.append(Qa_mean)
Qa_stadev.append(Qa_error/len(Qa_avg_v))
Qc_media.append(Qc_mean)
Qc_stadev.append(Qc_error/len(Qc_avg_v))


In [ ]:

Qc_err2.append(Qc_rms_v[Qc_avg_v.index(min(Qc_avg_v))])

Qa_err2.append(Qa_rms_v[Qa_avg_v.index(max(Qa_avg_v))])
Qa_media2.append(Qa_mean)
Qa_stadev2.append(Qa_error)
Qc_media2.append(Qc_mean)
Qc_stadev2.append(Qc_error)



In [ ]:
fig = plt.figure(figsize=(16,6))
measure1=[1]

print
#plt.plot_date(date_v,np.abs(Qa_avg_v))
#plt.plot_date(date_v,np.abs(Qc_avg_v))

plt.errorbar(Fiber_n,np.abs(Qa_media),yerr=Qa_stadev,fmt='o',color='r',label='Anode')
plt.errorbar(Fiber_n,np.abs(Qc_media),yerr=Qc_stadev,fmt='o',color='b',label='Cathode')

plt.xlim([0,10])

plt.ylim([0,1500])
plt.grid()
plt.legend(numpoints=1,fontsize=18, loc='upper left')
plt.xlabel('Fiber number',fontsize=20)
plt.ylabel('Amplitude',fontsize=20)
plt.title('Mean amplitude for every fiber',fontsize=30)
fig.savefig('measures_all_fibers_mean.png')
plt.show()

In [ ]:
print(Qa_media)
Qc_err=[9.6022923986750683, 5.8325176314028511, 10.222318669870331, 6.0536522432528352, 6.5575555847734108, 3.5454024301120541*10, 7.0583816119636991*10, 4.5520517291990181*10, 3.3307676655788532*10]
Qa_err=[8.0060645845182918, 4.8953870428852007, 9.4959801757909172, 5.5341915676395486, 7.0858739829084065, 3.4908543195412229*10, 6.5667202094266717*10, 4.9487367615887736*10, 3.1069041994329898*10]
Qc_stadev=[9.6022923986750683, 5.8325176314028511, 10.222318669870331, 6.0536522432528352, 29.351527634028137*10, 9.3841324261610168*10, 20.474941082658624*10, 12.801423709660302*10, 1.0500705505436407*10]
Qa_stadev=[8.0060645845182918, 4.8953870428852007, 9.4959801757909172, 5.5341915676395486, 27.963534753339598*10, 8.9332116055325823*10, 19.512890953012249*10, 12.160539801108003*10, 0.95748388359106451*10]
print(len(Qc_max), (Qc_stadev), len(Qa_max), (Qa_stadev), len(Fiber_n))

In [ ]:
Pulse_gen=[720,688,672,640,616,744,780,804,816,844,868,884,908,920,940,596,580,560,540,524,504,468,432,380,324,284,232,180,140,92,84]
Pulse_err=[4,4,4,4,4,4,4,4,8,4,4,4,4,8,4,4,4,8,4,4,8,4,8,4,4,4,8,4,4,4,4]
Pre_amp=[34,33.6,32.6,31.4,30.2,36.6,38.2,38.6,39.8,41.0,42.0,43,44,44.6,45.8,28.5,27.8,27,26.2,25.4,24.2,22.6,20.6,18.2,15.4,13.4,11,8.2,6.2,3.8,3.8]
Pre_amp_err=[0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.4,0.2,0.4,0.2,0.2,0.3,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2]
pre_line=np.polyfit(Pulse_gen,Pre_amp,1)
print(pre_line)
pre_line_plot = np.poly1d(pre_line)   
pre_residuals=np.array(Pre_amp)-np.array(pre_line_plot(Pulse_gen))
fig = plt.figure(figsize=(16,6))
frame1=fig.add_axes((.1,.3,.8,.6))

plt.errorbar(Pulse_gen,Pre_amp,xerr=Pulse_err,yerr=Pre_amp_err,fmt='o',color='r')
plt.errorbar(Pulse_gen,pre_line_plot(Pulse_gen),label='best fit = 0.04921818x - 0.53586418')

plt.grid()
plt.legend(numpoints=1,fontsize=18, loc='lower right')

plt.ylabel('Pre-amp signal [mV]',fontsize=20)
plt.title('Calibration of pre-amp',fontsize=30)
frame1.set_xticklabels([])
    
    
frame2=fig.add_axes((.1,.1,.8,.2))        
plt.plot(Pulse_gen,pre_residuals,'or')
plt.xlabel('Pulse generator signal [mV]',fontsize=20)
plt.grid()
fig.savefig('pre-amp.png')
plt.show()

In [ ]:
def integrale(values_vector, time_vector):
    baseline_anode,baseline_cathode=det_baseline(values_vector,time_vector)
    
    